# Setting

- coco_어쩌구 돼있는거 수정해야함.. >> 그냥 지우고 다시하죠

1. 데이터셋에서 캡션 로드 ????? 수정해야함
2. 각 캡션에서 15%의 단어 마스킹
3. 마스킹된 캡션을 UTF-8 바이트 시퀀스로 변환
4. 바이트와 특별 토큰을 모델 입력에 적합한 인덱스로 매핑
5. 입력 시퀀스, 어텐션 마스크, 타깃 레이블 준비

#### 논문 내용
1. 데이터 준비
    1. pretraining 데이터 준비
    2. 데이터 전처리 
        - SentencePiece: Devlin et al. (2019)에 따라 32,000개의 서브워드로 Vocabulary 생성
        - UTF-8 bytes: 전처리 없이 데이터를 UTF-8 바이트 시퀀스로 변환

2. Perceiver IO 구조 정의
    - 입력: SentencePiece 토큰 또는 UTF-8 bytes
    - 출력: Latent query를 사용해 Masked된 입력의 원래 값을 예측
    - Latent 크기: 256 
    - Processing layers: 26층 (SentencePiece 기준)

3. 토큰 마스크 생성
    - 입력 데이터 15% 랜덤 마스킹
    - BERT의 Masked Language Model(MLM) 방식 참고

In [ ]:
import json
import random
import numpy as np
from torch.utils.data import DataLoader, Dataset

SPECIAL_TOKENS = {
    '[PAD]': 256,
    '[MASK]': 257,
    '[CLS]': 258,
    '[SEP]': 259
}

In [ ]:
def process_caption(caption, mask_token='[MASK]', max_length=2048):

    # 단어로 분리 
    words = caption.strip().split()
    
    # 마스킹할 단어 수
    num_words_to_mask = max(1, int(0.15 * len(words)))
    
    # 마스킹할 단어의 인덱스 랜덤 선택
    masked_word_indices = random.sample(range(len(words)), num_words_to_mask)
    
    # 마스킹된 캡션 생성
    masked_words = words.copy()
    for idx in masked_word_indices:
        masked_words[idx] = mask_token  # 단어를 [MASK]로 대체
    
    # 원본 캡션 바이트 (타깃)
    target_text = ' '.join(words)
    target_bytes = target_text.encode('utf-8')
    
    # 마스킹된 캡션 바이트 (입력)
    masked_text = ' '.join(masked_words)
    masked_bytes = masked_text.encode('utf-8')
    
    # [MASK] 토큰의 바이트 표현
    mask_token_bytes = mask_token.encode('utf-8')

    # 시퀀스에서 부분 시퀀스의 모든 위치 찾기
    def find_subsequence_indices(sequence, subsequence):
        indices = []
        seq_len = len(sequence)
        sub_len = len(subsequence)
        i = 0
        while i <= seq_len - sub_len:
            if sequence[i:i+sub_len] == subsequence:
                indices.append((i, i+sub_len))
                i += sub_len
            else:
                i += 1
        return indices

    # 마스킹된 바이트에서 [MASK] 토큰의 위치 찾기
    mask_positions = find_subsequence_indices(masked_bytes, mask_token_bytes)

    # 바이트를 인덱스로 매핑 (0-255), 특별 토큰은 256-259
    input_indices = []
    i = 0
    while i < len(masked_bytes):
        # 현재 위치가 [MASK] 토큰의 시작인지 확인
        is_mask = False
        for start, end in mask_positions:
            if i == start:
                # [MASK] 토큰
                input_indices.append(SPECIAL_TOKENS['[MASK]'])  # [MASK]의 인덱스
                i = end  # 인덱스를 [MASK]의 끝으로 이동
                is_mask = True
                break
        if not is_mask:
            # 일반 바이트
            input_indices.append(masked_bytes[i])
            i += 1

    # max_length에 맞게 패딩 또는 잘라내기
    if len(input_indices) < max_length:
        input_indices += [SPECIAL_TOKENS['[PAD]']] * (max_length - len(input_indices))  # [PAD]로 패딩
    else:
        input_indices = input_indices[:max_length]

    # 레이블 생성: 마스킹되지 않은 위치는 -100, 마스킹된 위치는 타깃 바이트 인덱스
    labels = [-100] * len(input_indices)

    # 원본 캡션에서 단어의 바이트 위치 얻기
    def get_word_byte_positions(text):
        words = text.strip().split()
        positions = []
        pos = 0
        for word in words:
            word_bytes = word.encode('utf-8')
            word_len = len(word_bytes)
            positions.append((pos, pos + word_len))
            pos += word_len
            # 마지막 단어가 아니면 공백 문자 추가
            if word != words[-1]:
                pos += 1  # 공백 문자
        return positions

    # 타깃 바이트에서 단어의 위치 얻기
    word_positions = get_word_byte_positions(target_text)

    # 마스킹된 단어 위치에 레이블 설정
    for idx in masked_word_indices:
        start_pos, end_pos = word_positions[idx]
        # max_length를 초과하지 않도록 확인
        if end_pos > max_length:
            continue
        labels[start_pos:end_pos] = [target_bytes[i] for i in range(start_pos, end_pos)]

    # max_length에 맞게 레이블 패딩 또는 잘라내기
    if len(labels) < max_length:
        labels += [-100] * (max_length - len(labels))
    else:
        labels = labels[:max_length]

    # 어텐션 마스크 생성 (실제 토큰은 1, 패딩은 0)
    attention_mask = [1 if idx != SPECIAL_TOKENS['[PAD]'] else 0 for idx in input_indices]

    return input_indices, labels, attention_mask

# 캡션 로드 
captions = load_coco_captions('captions_train2017.json') # 고쳐야함

# 데이터셋 및 데이터로더 생성
dataset = CocoCaptionsDataset(captions)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 데이터로더 반복
for batch in dataloader:
    input_ids = batch['input_ids']
    labels = batch['labels']
    attention_mask = batch['attention_mask']
    break  